In [1]:
import torch
IsTPU = False
batch_size = 48
device = 'cuda'
dtype = torch.float32
speech_path = 'data/train_mp3s/'
data_path = 'data/train.csv'
num_workers = 16
clip = 3e-3
lr = 3e-5
output_path = ''
model_path = '/home/zhenlan/Desktop/Projects/Bengali ASR/T5Model_whisper'
background_audio = "/home/zhenlan/Desktop/Projects/Bengali ASR/ESC-50-master/audio"
background_music = "/home/zhenlan/Desktop/Projects/Bengali ASR/ESC-50-master/music"
add_data_path = 'dlsprint/train.csv'
add_audio = 'dlsprint/train_files/'
add_data_path2 = 'text_train'
add_audio2 = 'RESPIN/'

In [2]:
#from whisper_jax import FlaxWhisperForConditionalGeneration
from functions import *
import torch.nn as nn
from functools import partial
from transformers.modeling_outputs import BaseModelOutput
from transformers import WhisperForConditionalGeneration
from transformers.generation.configuration_utils import GenerationConfig
from transformers import T5ForConditionalGeneration
from transformers import AutoTokenizer
from torch.optim import AdamW,Adam,SGD
from torch.nn.utils import clip_grad_value_
from torch.cuda.amp import GradScaler
import time
import warnings
warnings.filterwarnings('ignore')

In [3]:
model = torch.load(model_path + '/'+ 't5.pth').to(device)

In [4]:
paras = list(model.encoder.fc1.parameters()) + list(model.encoder.fc2.parameters())
# paras = model.parameters()
opt = AdamW(paras,lr = lr,amsgrad=True)
#opt = Adam(paras,lr = lr)
#opt = SGD(paras,lr = lr)

In [5]:
for param in model.encoder.encoder.parameters():
    param.requires_grad = False
for param in model.decoder.parameters():
    param.requires_grad = False

In [6]:
epochs = 1
verbose = 100

tokenizer = AutoTokenizer.from_pretrained("csebuetnlp/banglat5")
feature_extractor = WhisperFeatureExtractor.from_pretrained("openai/whisper-large-v2")
text = pd.read_csv(data_path)
add_data = pd.read_csv(add_data_path)
# data source 2
df = pd.read_csv(add_data_path2,names=['code'])
df[['code','sentence']] = df["code"].str.split(" ", n=1, expand=True)
df.sentence = df.sentence.str.strip()

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This means that tokens that come after special tokens will not be properly handled. We recommend you to read the related pull request available at https://github.com/huggingface/transformers/pull/24565, and set the legacy attribute accordingly.


In [7]:
# "https://github.com/karoldvl/ESC-50/archive/master.zip"
from audiomentations import (
    AddBackgroundNoise,
    AddGaussianNoise,
    Compose,
    Gain,
    GainTransition,
    OneOf,
    PitchShift,
    PolarityInversion,
    TimeStretch,
    OneOf,
    )

# define augmentation
augmentation = Compose(
    [
        TimeStretch(min_rate=0.9, max_rate=1.1, p=1, leave_length_unchanged=False),
        Gain(min_gain_db=-6, max_gain_db=6, p=1),
        GainTransition(min_gain_db=-6, max_gain_db=6),
        PitchShift(min_semitones=-4, max_semitones=4, p=1),
        OneOf([AddBackgroundNoise(sounds_path=background_audio, min_snr_in_db=1.0, max_snr_in_db=5.0, noise_transform=PolarityInversion(), p=1.0),\
              AddBackgroundNoise(sounds_path=background_music, min_snr_in_db=1.0, max_snr_in_db=5.0, noise_transform=PolarityInversion(), p=1.0)]),
        AddGaussianNoise(min_amplitude=0.005, max_amplitude=0.015, p=1),
    ]
)


In [8]:
dataset1 = AudioDataset(text,speech_path,lambda x:x.id+'.mp3',\
                       augmentation,orig_sr=32000, target_sr=16000)
dataset2 = AudioDataset(add_data,add_audio,\
                        lambda x:x.path,augmentation,orig_sr=48000, target_sr=16000)
dataset3 = AudioDataset(df,add_audio2,\
                        lambda x:x.code.split('_')[-1] + '.wav',augmentation,orig_sr=16000, target_sr=16000)
# dataset1 = AudioDataset(text,speech_path,lambda x:x.id+'.mp3',\
#                        orig_sr=32000, target_sr=16000)
# dataset2 = AudioDataset(add_data,add_audio,\
#                         lambda x:x.path,orig_sr=48000, target_sr=16000)
# dataset3 = AudioDataset(df,add_audio2,\
#                         lambda x:x.code.split('_')[-1] + '.wav',orig_sr=16000, target_sr=16000)                        
dataset = Add2Data(dataset1,dataset2)
dataset = Add2Data(dataset,dataset3)
train_loader = DataLoader(dataset, batch_size=batch_size, shuffle=True, num_workers=num_workers, \
                            collate_fn=partial(collate_fn_pt,tokenizer=tokenizer,feature_extractor=feature_extractor,IsTrain=True))

In [9]:
# audio,input_ids,attention_mask = next(iter(train_loader))
# audio,input_ids,attention_mask = torch.asarray(audio,dtype=torch.float32,device=device),torch.asarray(input_ids,dtype=torch.long,device=device),torch.asarray(attention_mask,dtype=torch.float32,device=device)
# audio.shape,input_ids.shape
# audio,input_ids,attention_mask = jnp.array(audio,dtype=dtype),jnp.array(input_ids),jnp.array(attention_mask)

In [10]:
# using whisper V9 as encoder
model.train()
np.random.seed()
train_loss = 0
skip = 0 
# Training #
for j,(audio,input_ids,attention_mask) in enumerate(train_loader):
    if audio.shape[-1]>3000:
        skip+=1
        continue
    audio,input_ids,attention_mask = torch.asarray(audio,dtype=torch.float32,device=device),\
                                     torch.asarray(input_ids,dtype=torch.long,device=device),\
                                     torch.asarray(attention_mask,dtype=torch.float32,device=device)
    logits = model(input_ids=audio,decoder_input_ids=input_ids).logits
    loss = torch.sum(torch.nn.CrossEntropyLoss(reduction='none')(logits[:,0:-1].reshape(-1,32128),\
                                                                input_ids[:,1:].reshape(-1))*attention_mask[:,1:].reshape(-1))/torch.sum(attention_mask[:,1:])
    loss.backward()
    clip_grad_value_(paras,clip)
    opt.step()
    opt.zero_grad()
    train_loss += loss.item()
    if j>0 and j%verbose == 0:
        train_loss /= (verbose-skip)
        print(f"iterations:{j}, loss: {train_loss:.3f}")
        train_loss = 0
        skip = 0
    if j>2000:
        break
        # if j%50 ==0:
        #     torch.save(model, '/home/zhenlan/Desktop/Projects/Bengali ASR/T5Model/t5.pth')

iterations:100, loss: 6.414
iterations:200, loss: 6.215
iterations:300, loss: 6.197
iterations:400, loss: 6.172
iterations:500, loss: 6.146
iterations:600, loss: 6.137
iterations:700, loss: 6.096
iterations:800, loss: 6.092
iterations:900, loss: 6.097
iterations:1000, loss: 6.051
iterations:1100, loss: 6.043
iterations:1200, loss: 6.020
iterations:1300, loss: 6.032
iterations:1400, loss: 6.044
iterations:1500, loss: 6.032
iterations:1600, loss: 5.977
iterations:1700, loss: 5.990
iterations:1800, loss: 5.970
iterations:1900, loss: 5.950
iterations:2000, loss: 5.934


In [9]:
# using whisper V9 as encoder
model.train()
np.random.seed()
train_loss = 0
skip = 0 
# Training #
for j,(audio,input_ids,attention_mask) in enumerate(train_loader):
    if audio.shape[-1]>3000:
        skip+=1
        continue
    audio,input_ids,attention_mask = torch.asarray(audio,dtype=torch.float32,device=device),\
                                     torch.asarray(input_ids,dtype=torch.long,device=device),\
                                     torch.asarray(attention_mask,dtype=torch.float32,device=device)
    logits = model(input_ids=audio,decoder_input_ids=input_ids).logits
    loss = torch.sum(torch.nn.CrossEntropyLoss(reduction='none')(logits[:,0:-1].reshape(-1,32128),\
                                                                input_ids[:,1:].reshape(-1))*attention_mask[:,1:].reshape(-1))/torch.sum(attention_mask[:,1:])
    loss.backward()
    clip_grad_value_(paras,clip)
    opt.step()
    opt.zero_grad()
    train_loss += loss.item()
    if j>0 and j%verbose == 0:
        train_loss /= (verbose-skip)
        print(f"iterations:{j}, loss: {train_loss:.3f}")
        train_loss = 0
        skip = 0
    if j>4000:
        break
        # if j%50 ==0:
        #     torch.save(model, '/home/zhenlan/Desktop/Projects/Bengali ASR/T5Model/t5.pth')

In [10]:
# use_amp = True
# model.train()
# np.random.seed()
# train_loss = 0
# skip = 0
# scaler = torch.cuda.amp.GradScaler(enabled=use_amp)
# # Training #
# for j,(audio,input_ids,attention_mask) in enumerate(train_loader):
#     if audio.shape[-1]>3000:
#         skip+=1
#         continue
#     audio,input_ids,attention_mask = torch.asarray(audio,device=device),\
#                                      torch.asarray(input_ids,device=device),\
#                                      torch.asarray(attention_mask,device=device)
#     with torch.autocast(device_type=device, dtype=torch.float16, enabled=use_amp):
#         logits = model(input_ids=audio,decoder_input_ids=input_ids).logits
#         loss = torch.sum(torch.nn.CrossEntropyLoss(reduction='none')(logits[:,0:-1].reshape(-1,32128),\
#                                                                      input_ids[:,1:].reshape(-1))*attention_mask[:,1:].reshape(-1)/torch.sum(attention_mask[:,1:]))
#     scaler.scale(loss).backward()
#     scaler.unscale_(opt)
#     clip_grad_value_(paras,clip)
#     scaler.step(opt)
#     scaler.update()
#     opt.zero_grad()
#     #print(torch.any(torch.isnan(logits)),scaler.get_scale())
#     train_loss += loss.item()
#     if j>0 and j%1 == 0:
#         train_loss /= (verbose-skip)
#         print(f"iterations:{j}, loss: {train_loss:.3f}")
#         train_loss = 0
#         skip = 0
#     if j>2000:
#         break

In [11]:
torch.save(model, model_path + '/t5.pth')